In [ ]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 80.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

In [4]:
tesla = yf.Ticker("TSLA")

In [5]:
tesla_data = tesla.history(period="max")

In [6]:
tesla_data.reset_index(inplace=True)
print(tesla_data.head())

                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


In [10]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

In [11]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_data, "html5lib")

In [12]:
import pandas as pd
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_data, "html5lib")

# Find the table with the title "Tesla Annual Revenue"
table = soup.find('table', class_='historical_data_table table table-striped mt-5')

# Extract the table data
data = []
if table:
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

# Create the DataFrame
tesla_revenue = pd.DataFrame(data, columns=['Date', 'Revenue'])

# Clean the Revenue column
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

print(tesla_revenue.head())

Empty DataFrame
Columns: [Date, Revenue]
Index: []


In [22]:
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf
import requests
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Set plotly renderer
pio.renderers.default = "iframe"

# Define the graph function
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    # Ensure the 'Revenue' column is cleaned before converting to float
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

# Fetch Tesla stock data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)

# Fetch Tesla revenue data from the provided URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

# Access the second tbody element which contains the annual revenue data
table = soup.find_all("tbody")[1]

# Extract the table data
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# Create the DataFrame
tesla_revenue = pd.DataFrame(data, columns=['Date', 'Revenue'])

# Clean the Revenue column - Ensure this is done correctly and only once
# Remove commas and dollar signs using regex
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(r',|\$', "", regex=True)
# Remove any empty strings after cleaning
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

# Convert the 'Revenue' column to numeric after cleaning
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"])

# Now call the make_graph function with the cleaned data
make_graph(tesla_data, tesla_revenue,"tesla")

Output hidden; open in https://colab.research.google.com to view.

In [14]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

In [15]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [16]:
print(tesla_revenue.tail())

          Date Revenue
48  2010-09-30     $31
49  2010-06-30     $28
50  2010-03-31     $21
52  2009-09-30     $46
53  2009-06-30     $27


In [23]:
gme = yf.Ticker("GME")

In [ ]:
gme_data = gme.history(period="max")

In [24]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_2 = requests.get(url).text

In [25]:
from bs4 import BeautifulSoup

soup_2 = BeautifulSoup(html_data_2, "html5lib")

In [26]:
import pandas as pd
from bs4 import BeautifulSoup

soup_2 = BeautifulSoup(html_data_2, "html5lib")

# Find the table with the title "GameStop Annual Revenue" or similar identifier
# Based on the provided code, it seems the relevant table might be the second tbody
table = soup_2.find_all("tbody")[1]

# Extract the table data
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# Create the DataFrame
gme_revenue = pd.DataFrame(data, columns=['Date', 'Revenue'])

# Clean the Revenue column
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [27]:
import pandas as pd
from bs4 import BeautifulSoup

soup_2 = BeautifulSoup(html_data_2, "html5lib")

# Find the table with the title "GameStop Annual Revenue" or similar identifier
# Based on the provided code, it seems the relevant table might be the second tbody
table = soup_2.find_all("tbody")[1]

# Extract the table data
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# Create the DataFrame
gme_revenue = pd.DataFrame(data, columns=['Date', 'Revenue'])

# Clean the Revenue column
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [28]:
print(gme_revenue.tail())

          Date Revenue
57  2006-01-31    1667
58  2005-10-31     534
59  2005-07-31     416
60  2005-04-30     475
61  2005-01-31     709


In [30]:
gme_data = gme.history(period="max")

In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf
import requests
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Set plotly renderer
pio.renderers.default = "iframe"

# Define the graph function
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    # Ensure the 'Revenue' column is cleaned before converting to float
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

# Fetch Tesla stock data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)

# Fetch Tesla revenue data from the provided URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

# Access the second tbody element which contains the annual revenue data
table = soup.find_all("tbody")[1]

# Extract the table data
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# Create the DataFrame
tesla_revenue = pd.DataFrame(data, columns=['Date', 'Revenue'])

# Clean the Revenue column - Ensure this is done correctly and only once
# Remove commas and dollar signs using regex
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(r',|\$', "", regex=True)
# Remove any empty strings after cleaning
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

# Convert the 'Revenue' column to numeric after cleaning
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"])

# Fetch GME stock data
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
# Reset the index to turn the date index into a 'Date' column
gme_data.reset_index(inplace=True)


# Fetch GME revenue data from the provided URL
url_2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_2 = requests.get(url_2).text
soup_2 = BeautifulSoup(html_data_2, "html5lib")

# Find the table with the title "GameStop Annual Revenue" or similar identifier
# Based on the provided code, it seems the relevant table might be the second tbody
table_2 = soup_2.find_all("tbody")[1]

# Extract the table data
data_2 = []
for row in table_2.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data_2.append([ele for ele in cols if ele]) # Get rid of empty values

# Create the DataFrame
gme_revenue = pd.DataFrame(data_2, columns=['Date', 'Revenue'])

# Clean the Revenue column - Ensure this is done correctly and only once
# Remove commas and dollar signs using regex
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(r',|\$', "", regex=True)
# Remove any empty strings after cleaning
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

# Convert the 'Revenue' column to numeric after cleaning
gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"])

# Now call the make_graph function with the cleaned data
make_graph(gme_data, gme_revenue, 'gme')

Output hidden; open in https://colab.research.google.com to view.